# Arboles de Decisión


In [1]:
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import json
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from dataframe_builder import DataFrameBuilder
# Ignorar Warning de sklearn (no es importante)
import warnings
warnings.filterwarnings('ignore',
                        message='Changing the shape of non-C contiguous array')


print "Leyendo archivos..."


builder = DataFrameBuilder()
df = builder.build(ham=json.load(open('../data/ham_dev.json')), spam=json.load(open('../data/spam_dev.json')))


Leyendo archivos...


Saco la columna de text porque ocupa mucho espacio

In [2]:
import sys
df.drop('text', axis=1, inplace=True)
sys.getsizeof(df) / (2.0**20)

17.959059715270996


Algunas observaciones previas:

El árbol que se genera no es usando ID3 sino CART. Una pequeña introducción
se puede ver en

http://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart

No está muy completo el asunto ahí, habría que hurgar algún paper extra

Opciones:

- criterion: gini o entropy. Son dos medidas distintas
- splitter: No estoy muy seguro
- max_depth: máxima profundidad del árbol
- max_features: qué porcentaje de variables tomo a la hora de partir un nodo.
  Estas variables se eligen aleatoriamente
- min_samples_split: cuántos elementos tengo que tener en un nodo para decidir
  partirlo

In [5]:
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

options = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': range(2, 52, 8),
    'max_features': np.arange(0.1, 1.0, 0.2),
    'min_samples_split': range(2, 102, 10),
}

# Preparo data para clasificar
X = df[builder.list_of_attributes].values
y = df['class'] == 'spam'

for scoring in ['precision', 'accuracy', 'f1']:
    clf = DecisionTreeClassifier()

    print("=" * 80 + "\n")
    print("Scoring {}".format(scoring))

    #search = GridSearchCV(clf, scoring=scoring, param_grid=options, n_jobs=4)
    search = RandomizedSearchCV(clf, scoring=scoring, param_distributions=options, n_jobs=4, n_iter=200)
    
    search.fit(X, y)

    print "Mejor combinación: {}".format(search.best_params_)
    print "Mejor valor: {}".format(search.best_score_)



Scoring precision
Mejor combinación: {'min_samples_split': 92, 'splitter': 'random', 'criterion': 'entropy', 'max_depth': 2, 'max_features': 0.90000000000000013}
Mejor valor: 0.970907736081

Scoring accuracy
Mejor combinación: {'min_samples_split': 2, 'splitter': 'best', 'criterion': 'entropy', 'max_depth': 42, 'max_features': 0.90000000000000013}
Mejor valor: 0.946873263782

Scoring f1
Mejor combinación: {'min_samples_split': 2, 'splitter': 'best', 'criterion': 'entropy', 'max_depth': 34, 'max_features': 0.50000000000000011}
Mejor valor: 0.947818524103
